In [ ]:
########code for transfor learning using resnet50

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model
import numpy as np

# Path to the dataset directory
train_dataset_path = '/content/gdrive/My Drive/race classification/Dataset/train'
test_dataset_path = '/content/gdrive/My Drive/race classification/Dataset/test'
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
# Load the dataset
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dataset_path,
    labels='inferred',  # Automatically infer labels from folder names
    label_mode='int',   # Return integer labels (can also be 'categorical' or 'binary')
    image_size=(224, 224),  # Resize all images to this size
    batch_size=4,      # Number of images in each batch
    shuffle=True,       # Shuffle the data
    seed=42,            # Seed for reproducibility
)



# Load the validation dataset
test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dataset_path,
    labels='inferred',
    label_mode='int',
    image_size=(224, 224),
    batch_size=1,
    shuffle=False,

)

from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load pre-trained ResNet50 model without the top layers (classification layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all layers except the last few
for layer in base_model.layers:
    layer.trainable = False

# Unfreeze the last few layers (e.g., the last 4 layers)
for layer in base_model.layers[-8:]:
    layer.trainable = True

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling to reduce the dimensionality
x = Dense(1024, activation='relu')(x)  # Fully connected layer
predictions = Dense(2, activation='softmax')(x)  # Output layer (Asian, non-Asian)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',  # Use 'binary_crossentropy' if using two output neurons
              metrics=['accuracy'])




checkpoint = ModelCheckpoint(
    '/content/gdrive/My Drive/race classification/best_model_resnet-V2[last 8 layers].keras',  # File path where the model will be saved
    monitor='val_accuracy',  # Metric to monitor
    save_best_only=True,  # Save only the best model (with the lowest validation loss)
    mode='max',  # Mode is 'min' for validation loss since lower is better
    verbose=1  # Print messages when saving the model
)
# Train the model
model.fit(
    train_dataset,
    validation_data=test_dataset,
    callbacks=[checkpoint],  # Include the ModelCheckpoint callback
    epochs=300
)



In [ ]:


from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

resnet_model=load_model('/content/gdrive/My Drive/race classification/best_model_resnet-V2[last 8 layers].keras')

resnet_pred = resnet_model.predict(test_dataset, verbose=1)

final_predictions = np.argmax(resnet_pred, axis=1)
true_labels = []
for _, labels in test_dataset:
    true_labels.extend(labels.numpy())
true_labels = np.array(true_labels)

accuracy = accuracy_score(true_labels, final_predictions)
print(f"\n✅ Ensemble Accuracy: {accuracy * 100:.2f}%")

print("\n📋 Classification Report:")
print(classification_report(true_labels, final_predictions, target_names=test_dataset.class_names))

print("\n📊 Confusion Matrix:")
print(confusion_matrix(true_labels, final_predictions))


# Model summary
vgg_model.summary()

In [ ]:
############  FaceNet embeddings, MTCNN,  Logistic Regression



from skimage.feature import local_binary_pattern
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model
import numpy as np


import os
import cv2
import torch
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from facenet_pytorch import InceptionResnetV1, MTCNN

# Path to the dataset directory
train_dataset_path = '/content/gdrive/My Drive/race classification/Dataset/train'
test_dataset_path = '/content/gdrive/My Drive/race classification/Dataset/test'
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
# Load the dataset
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dataset_path,
    labels='inferred',  # Automatically infer labels from folder names
    label_mode='int',   # Return integer labels (can also be 'categorical' or 'binary')
    image_size=(224, 224),  # Resize all images to this size
    batch_size=4,      # Number of images in each batch
    shuffle=True,       # Shuffle the data
    seed=42,            # Seed for reproducibility
)



# Load the validation dataset
test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dataset_path,
    labels='inferred',
    label_mode='int',
    image_size=(224, 224),
    batch_size=1,
    shuffle=False,

)
# Load FaceNet and MTCNN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
facenet = InceptionResnetV1(pretrained='vggface2').eval().to(device)
mtcnn = MTCNN(image_size=160, device=device)

# Function to extract embeddings and labels from tf.data.Dataset
def extract_embeddings(dataset):
    embeddings, labels = [], []
    for batch_images, batch_labels in dataset:
        for img_tensor, label in zip(batch_images, batch_labels):
            img = tf.image.resize(img_tensor, (160, 160)).numpy().astype("uint8")
            face = mtcnn(img)
            if face is not None:
                with torch.no_grad():
                    emb = facenet(face.unsqueeze(0).to(device)).cpu().numpy().flatten()
                    embeddings.append(emb)
                    labels.append(label.numpy())
    return np.array(embeddings), np.array(labels)
# === Extract embeddings ===
print("Extracting embeddings from training dataset...")
X_train, y_train = extract_embeddings(train_dataset)

print("Extracting embeddings from test dataset...")
X_test, y_test = extract_embeddings(test_dataset)

# === Train logistic regression ===
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# === Evaluate ===
y_pred = clf.predict(X_test)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
############  FaceNet embeddings, MTCNN,  MLP


from skimage.feature import local_binary_pattern
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import numpy as np

import os
import cv2
import torch
import numpy as np
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from facenet_pytorch import InceptionResnetV1, MTCNN
from tensorflow.keras.layers import Input, Dense

# Path to the dataset directory
train_dataset_path = '/content/gdrive/My Drive/race classification/Dataset/train'
test_dataset_path = '/content/gdrive/My Drive/race classification/Dataset/test'
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# Load the dataset
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dataset_path,
    labels='inferred',
    label_mode='int',
    image_size=(224, 224),
    batch_size=4,
    shuffle=True,
    seed=42,
)

# Load the validation dataset
test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dataset_path,
    labels='inferred',
    label_mode='int',
    image_size=(224, 224),
    batch_size=1,
    shuffle=False,
)

# Load FaceNet and MTCNN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
facenet = InceptionResnetV1(pretrained='vggface2').eval().to(device)
mtcnn = MTCNN(image_size=160, device=device)

# Function to extract embeddings and labels from tf.data.Dataset
def extract_embeddings(dataset):
    embeddings, labels = [], []
    for batch_images, batch_labels in dataset:
        for img_tensor, label in zip(batch_images, batch_labels):
            img = tf.image.resize(img_tensor, (160, 160)).numpy().astype("uint8")
            face = mtcnn(img)
            if face is not None:
                with torch.no_grad():
                    emb = facenet(face.unsqueeze(0).to(device)).cpu().numpy().flatten()
                    embeddings.append(emb)
                    labels.append(label.numpy())
    return np.array(embeddings), np.array(labels)

# === Extract embeddings ===
print("Extracting embeddings from training dataset...")
X_train, y_train = extract_embeddings(train_dataset)

print("Extracting embeddings from test dataset...")
X_test, y_test = extract_embeddings(test_dataset)

# Convert labels to categorical
num_classes = len(np.unique(y_train))
y_train_cat = to_categorical(y_train, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

# === Define MLP model ===
model = Sequential([
    Input(shape=(X_train.shape[1],)),  # Proper input layer
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# === Train MLP ===
model.fit(X_train, y_train_cat, epochs=20, batch_size=32, validation_split=0.1)

# === Evaluate on test data ===
test_loss, test_acc = model.evaluate(X_test, y_test_cat)
print(f"Test Accuracy: {test_acc:.4f}")

# === Predict and classification report ===
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))